## Day 24: Blizzard Basin

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2022/tree/HEAD/notebooks/day/puzzle24.ipynb)

### Part 1

In [93]:
#!value --name sampleRaw 
#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#

In [94]:
#load "../common/common.fsx"
open System.Collections.Generic

type Facing = Point

type Blizzard = {Coordinates: Point; Facing: Facing; }

type Blizzards = {Map: Dictionary<Point, Blizzard array>; Min: Point; Max: Point}
with member this.Size = this.Max - this.Min + Point(1,1)
     member this.InRange (p : Point) = 
            Point.x p >= Point.x this.Min
            && Point.x p <= Point.x this.Max
            && Point.y p >= Point.y this.Min
            && Point.y p <= Point.y this.Max

let down : Facing = Point (1,0)
let up : Facing = Point (-1,0)
let right : Facing = Point (0,1)
let left : Facing = Point (0,-1)

In [95]:
let parse s = 
    let m = 
        Pattern1.read Seq.toArray s
        |> array2D
    let dic = 
        m
        |> Array2D.indexed
        |> Array2D.toSeq
        |> Seq.choose (fun (p,c) -> 
            match c with 
            | '>' -> Some (p,right)
            | 'v' -> Some (p,down)
            | '<'-> Some (p,left)
            | '^' -> Some (p,up)
            | _ -> None 
        )
        |> Seq.map (fun (p, v) -> KeyValuePair.Create(Point p, Array.singleton {Coordinates = Point p; Facing = v}))
        |> Dictionary<_, _>
    { Map = dic; Min = Point(1,1); Max = Point(Array2D.length1 m - 2, Array2D.length2 m - 2) }


In [96]:
#!share sampleRaw --from value

let sampleBlizzards = parse sampleRaw

To model wind we make use of the previously defined Euclidean remainder operator `%%` and just build a new dictionary.

In [97]:
let wind' blizzards = 
    let bs = blizzards.Map.Values |> Seq.collect id
    let zero = blizzards.Min
    let size = blizzards.Size
    let pmod (Point(mx,my)) (Point(x,y)) = 
        Point(x %% mx, y %% my)
    let newMap = 
        bs |> Seq.map (fun b ->
            let newCoordinates = b.Coordinates + b.Facing
            {
                Coordinates = ((newCoordinates - zero) |> pmod size) + zero 
                Facing = b.Facing
            }
        )
        |> Seq.group (fun b -> b.Coordinates) Seq.toArray
        |> Seq.map (KeyValuePair.Create)
        |> Dictionary<_, _>
    {blizzards with Map = newMap}

However, considering the blizzards state at some moment of time, it is worth noting that the state is going to repeat every $\mathrm{LCM} \left(width, height \right)$ steps. The code below relies on funcions `memoize`, `lcm` from the `common.fsx` script.

A memoized function is available via the generator function `wind()` to make the cache controllable.

In [98]:
type State = { Blizzards : Blizzards; Position : Point; Step : int }

let windF() = 
    memoize 
        (fun state -> wind' state.Blizzards) 
        (fun state -> state.Step % lcm (Point.x state.Blizzards.Size) (Point.y state.Blizzards.Size))


This notebook relies on a breadth-first search algorithm implementation from the `bfs.fsx` script file. 

In this problem we have a graph of decicions taken during each minute, including waiting, if possible. The graph is traversed in search of the goal.

In [99]:
#load "../common/bfs.fsx"

open Bfs
open Bfs.Custom

let goalTarget : Target<State> =
    fun state -> 
        state.Position = state.Blizzards.Max + down

let adjF() : Adjacency<State> =
    let wind = windF()
    fun state ->
        [
            let afterWind = wind state
            
            let options =
                [ up; down; left; right]
                |> List.map ((+)state.Position)
                |> List.filter (fun p -> state.Blizzards.InRange(p)
                                        || p = state.Blizzards.Max + down
                                        || p = state.Blizzards.Min + up
                                )
                |> List.filter (fun p -> not <| afterWind.Map.ContainsKey(p))
                |> List.map(fun p -> { Position = p; Blizzards = afterWind; Step = state.Step + 1})
            
            yield! options

            let canWait = not <| afterWind.Map.ContainsKey(state.Position)
            if canWait
            then
                yield {state with Blizzards = afterWind; Step = state.Step + 1}
        ]

let settings =
    { VisitedKey = fun state -> state.Position, state.Step % lcm (Point.x state.Blizzards.Size) (Point.y state.Blizzards.Size)}

let findFoundPath adj target start = 
    let (Found(path)) = findPath settings {Adjacency = adj} start target
    path

In [100]:
let sampleStart = {
    Position = sampleBlizzards.Min + up
    Blizzards = sampleBlizzards
    Step = 0
}

let private sampleResult = 
    findFoundPath (adjF()) goalTarget sampleStart 
    |> List.head
sampleResult.Step

18

For the actual input:

In [101]:
#!value --name actualRaw --from-file ./data_actual.txt

In [102]:
#!share actualRaw --from value

let actualBlizzards = parse actualRaw

let actualStart = {
    Position = actualBlizzards.Min + up
    Blizzards = actualBlizzards
    Step = 0
}

In [103]:
#!time 

let private actualResult = 
    findFoundPath (adjF()) goalTarget actualStart
    |> List.head
actualResult.Step

225

Wall time: 4081.4366ms

### Part 2

For part 2, the trips are calculated one by one with intermediate results being piped from search to search. 

The `adjF` function is called once to reuse memoization caches.

In [104]:
let startTarget : Target<State> =
    fun state -> 
        state.Position = state.Blizzards.Min + up

let private adj = adjF()
let private sampleResult = 
    sampleStart
    |> findFoundPath adj goalTarget
    |> List.head
    |> findFoundPath adj startTarget
    |> List.head
    |> findFoundPath adj goalTarget
    |> List.head

sampleResult.Step

54

For the actual input:

In [105]:
#!time

let private adj = adjF()
let private actualResult = 
    actualStart
    |> findFoundPath adj goalTarget
    |> List.head
    |> findFoundPath adj startTarget
    |> List.head
    |> findFoundPath adj goalTarget
    |> List.head

actualResult.Step

711

Wall time: 16244.1936ms